### CRAB AGE PREDICTOR 

In [66]:
from warnings import filterwarnings
filterwarnings('ignore')

### READING DATASET

In [67]:
import pandas as pd
df= pd.read_csv('train.csv')
df.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74051 entries, 0 to 74050
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              74051 non-null  int64  
 1   Sex             74051 non-null  object 
 2   Length          74051 non-null  float64
 3   Diameter        74051 non-null  float64
 4   Height          74051 non-null  float64
 5   Weight          74051 non-null  float64
 6   Shucked Weight  74051 non-null  float64
 7   Viscera Weight  74051 non-null  float64
 8   Shell Weight    74051 non-null  float64
 9   Age             74051 non-null  int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 5.6+ MB


In [69]:
df.isna().sum()

id                0
Sex               0
Length            0
Diameter          0
Height            0
Weight            0
Shucked Weight    0
Viscera Weight    0
Shell Weight      0
Age               0
dtype: int64

### SEPERATE X AND Y FEATURE

In [70]:
X = df.drop(columns=['id','Age'])
Y = df[['Age']]

In [71]:
X.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928
1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940
2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133
3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885
4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395


In [72]:
Y.head()

,Age
0,9
1,8
2,9
3,11
4,8


### SEPERATE CAT AND CON FEATURE

In [73]:
cat = list(X.columns[X.dtypes=='object'])
con = list(X.columns[X.dtypes!='object'])

In [74]:
cat

['Sex']

In [75]:
con

['Length',
 'Diameter',
 'Height',
 'Weight',
 'Shucked Weight',
 'Viscera Weight',
 'Shell Weight']

### 1ST PIPELINE

In [76]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer

In [77]:
num_pipe= Pipeline(steps=[('impute',SimpleImputer(strategy='median')),
                          ('scaler',StandardScaler())])
cat_pipe= Pipeline(steps=[('impute',SimpleImputer(strategy='most_frequent')),
                          ('ord',OrdinalEncoder())])
pre= ColumnTransformer([('num',num_pipe,con),
                        ('cat',cat_pipe,cat)])

In [78]:
x_pre= pre.fit_transform(X)
x_pre

array([[ 0.72123811,  0.63398203,  0.29239968, ...,  0.56918615,
         0.45337609,  1.        ],
       [-0.75571183, -0.84035601, -0.79416309, ..., -0.97887971,
        -0.92678786,  1.        ],
       [ 0.24340136,  0.37070738,  0.29239968, ...,  0.17836296,
        -0.01722423,  2.        ],
       ...,
       [ 0.590919  ,  0.73929189,  0.69986072, ...,  0.88895057,
         0.37823822,  0.        ],
       [-0.36475449, -0.26115178, -0.38670205, ..., -0.76570342,
        -0.68160114,  1.        ],
       [-1.40730739, -1.47221517, -1.60908517, ..., -1.44076165,
        -1.4013428 ,  1.        ]])

In [79]:
cols= pre.get_feature_names_out()
cols

array(['num__Length', 'num__Diameter', 'num__Height', 'num__Weight',
       'num__Shucked Weight', 'num__Viscera Weight', 'num__Shell Weight',
       'cat__Sex'], dtype=object)

In [80]:
x_pre=pd.DataFrame(x_pre,columns=cols)
x_pre

,num__Length,num__Diameter,num__Height,num__Weight,num__Shucked Weight,num__Viscera Weight,num__Shell Weight,cat__Sex
0,0.721238,0.633982,0.292400,0.441804,0.467188,0.569186,0.453376,1.0
1,-0.755712,-0.840356,-0.794163,-1.025198,-0.993688,-0.978880,-0.926788,1.0
2,0.243401,0.370707,0.292400,0.110076,0.219924,0.178363,-0.017224,2.0
3,1.329394,1.634426,1.650603,2.156483,1.824616,2.124622,2.308095,0.0
4,-0.234435,-0.050532,-0.115061,-0.007598,0.333464,-0.197233,-0.214955,1.0
...,...,...,...,...,...,...,...,...
74046,1.199075,1.002567,0.971501,2.156483,1.882648,1.899000,1.564626,0.0
74047,-0.842591,-0.682391,-0.794163,-1.022957,-1.029011,-0.989031,-0.887242,1.0
74048,0.590919,0.739292,0.699861,0.482150,0.391495,0.888951,0.378238,0.0
74049,-0.364754,-0.261152,-0.386702,-0.523122,-0.201434,-0.765703,-0.681601,1.0


In [81]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
Lr = LinearRegression()

In [82]:
sel= SequentialFeatureSelector(Lr,direction='backward',n_jobs=-1)
sel.fit_transform(x_pre,Y)

array([[ 0.63398203,  0.44180441,  0.4671879 ,  0.45337609],
       [-0.84035601, -1.02519817, -0.99368761, -0.92678786],
       [ 0.37070738,  0.11007579,  0.219924  , -0.01722423],
       ...,
       [ 0.73929189,  0.48214978,  0.39149487,  0.37823822],
       [-0.26115178, -0.52312242, -0.20143388, -0.68160114],
       [-1.47221517, -1.42304837, -1.43270719, -1.4013428 ]])

In [83]:
sel_cols=sel.get_feature_names_out()
sel_cols

array(['num__Diameter', 'num__Weight', 'num__Shucked Weight',
       'num__Shell Weight'], dtype=object)

In [84]:
imp_cols=[]
for i in sel_cols:
    s= i.split('__')[1]
    imp_cols.append(s)

In [85]:
imp_cols

['Diameter', 'Weight', 'Shucked Weight', 'Shell Weight']

In [86]:
X_sel=X[imp_cols]

In [87]:
X_cat=list(X_sel.columns[X_sel.dtypes=='object'])
X_con=list(X_sel.columns[X_sel.dtypes!='object'])

### CREATE FINAL PIPELINE

In [88]:
num_pipe1= Pipeline(steps=[('impute',SimpleImputer(strategy='median')),
                          ('scaler',StandardScaler())])
cat_pipe1= Pipeline(steps=[('impute',SimpleImputer(strategy='most_frequent')),
                          ('ohe',OneHotEncoder(handle_unknown='ignore'))])
pre1= ColumnTransformer([('num',num_pipe1,X_con),
                        ('cat',cat_pipe1,X_cat)])

In [89]:
X_sel_pre=pre1.fit_transform(X_sel)
X_sel_pre

array([[ 0.63398203,  0.44180441,  0.4671879 ,  0.45337609],
       [-0.84035601, -1.02519817, -0.99368761, -0.92678786],
       [ 0.37070738,  0.11007579,  0.219924  , -0.01722423],
       ...,
       [ 0.73929189,  0.48214978,  0.39149487,  0.37823822],
       [-0.26115178, -0.52312242, -0.20143388, -0.68160114],
       [-1.47221517, -1.42304837, -1.43270719, -1.4013428 ]])

In [90]:
cols2= pre1.get_feature_names_out()
cols2

array(['num__Diameter', 'num__Weight', 'num__Shucked Weight',
       'num__Shell Weight'], dtype=object)

In [91]:
X_sel_pre=pd.DataFrame(X_sel_pre,columns=cols2)
X_sel_pre

,num__Diameter,num__Weight,num__Shucked Weight,num__Shell Weight
0,0.633982,0.441804,0.467188,0.453376
1,-0.840356,-1.025198,-0.993688,-0.926788
2,0.370707,0.110076,0.219924,-0.017224
3,1.634426,2.156483,1.824616,2.308095
4,-0.050532,-0.007598,0.333464,-0.214955
...,...,...,...,...
74046,1.002567,2.156483,1.882648,1.564626
74047,-0.682391,-1.022957,-1.029011,-0.887242
74048,0.739292,0.482150,0.391495,0.378238
74049,-0.261152,-0.523122,-0.201434,-0.681601


### TRAIN TEST SPLIT

In [92]:
from sklearn.model_selection import train_test_split

In [93]:
xtrain,xtest,ytrain,ytest= train_test_split(X_sel_pre,Y,test_size=0.2,random_state=21)

In [94]:
xtrain.shape

(59240, 4)

In [95]:
xtest.shape

(14811, 4)

### EVALUATING THE MODEL

In [96]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

from sklearn.neighbors import KNeighborsRegressor

In [97]:
dct={'Linear Regression':LinearRegression(),
     'Decsion tree':DecisionTreeRegressor(),
     'Random forest':RandomForestRegressor(),
     'Gradient boosting':GradientBoostingRegressor(),
     'KNN':KNeighborsRegressor()
     }

In [98]:
dct.items()

dict_items([('Linear Regression', LinearRegression()), ('Decsion tree', DecisionTreeRegressor()), ('Random forest', RandomForestRegressor()), ('Gradient boosting', GradientBoostingRegressor()), ('KNN', KNeighborsRegressor())])

### EVALUATION MODEL

In [99]:
from sklearn.metrics import mean_absolute_error


In [100]:
tr=[]
ts=[]


In [101]:
for name,model in dct.items():
    model.fit(xtrain,ytrain)
    ypred_tr=model.predict(xtrain)
    ypred_ts=model.predict(xtest)
    f1_tr=mean_absolute_error(ytrain,ypred_tr)
   
    

    f1_ts=mean_absolute_error(ytest,ypred_ts)
    tr.append(f1_tr)
    ts.append(f1_ts)
  
    print('\n---------------------------------------\n')
    print(f'Model Name : {name}\n')
    print(f'Training mean absolute error : {f1_tr:.4f}')
 
    print(f'Testing mean absolute error : {f1_ts:.4f}')


---------------------------------------

Model Name : Linear Regression

Training mean absolute error : 1.5148
Testing mean absolute error : 1.5424



---------------------------------------

Model Name : Decsion tree

Training mean absolute error : 0.0892
Testing mean absolute error : 2.0251

---------------------------------------

Model Name : Random forest

Training mean absolute error : 0.6096
Testing mean absolute error : 1.5513

---------------------------------------

Model Name : Gradient boosting

Training mean absolute error : 1.4347
Testing mean absolute error : 1.4665

---------------------------------------

Model Name : KNN

Training mean absolute error : 1.2775
Testing mean absolute error : 1.5806


### Selecting the Linear regression model

In [102]:
from sklearn.linear_model import Ridge,Lasso
model2=Ridge()
model2.fit(xtrain,ytrain)


Ridge()

In [103]:
model2.score(xtrain,ytrain)

0.5340875760681467

In [104]:
model2.score(xtest,ytest)

0.5180000174161186

In [105]:
import numpy as np

In [106]:
alpha= np.arange(0.1,50,0.1)
params= {'alpha':alpha}

### RIDGE TUNNING

In [107]:
from sklearn.model_selection import RandomizedSearchCV
rcv = RandomizedSearchCV(model2,param_distributions=params,cv=5,scoring='neg_mean_absolute_error')
rcv.fit(xtrain,ytrain)

RandomizedSearchCV(cv=5, estimator=Ridge(),
                   param_distributions={'alpha': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,
        1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,
        2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,
        3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,
        4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,  5.5,
        5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,  6.6,
        6.7,  6.8,  6.9,  7....
       43. , 43.1, 43.2, 43.3, 43.4, 43.5, 43.6, 43.7, 43.8, 43.9, 44. ,
       44.1, 44.2, 44.3, 44.4, 44.5, 44.6, 44.7, 44.8, 44.9, 45. , 45.1,
       45.2, 45.3, 45.4, 45.5, 45.6, 45.7, 45.8, 45.9, 46. , 46.1, 46.2,
       46.3, 46.4, 46.5, 46.6, 46.7, 46.8, 46.9, 47. , 47.1, 47.2, 47.3,
       47.4, 47.5, 47.6, 47.7, 47.8, 47.9, 48. , 48.1, 48.2, 48.3, 48.4,
       48.5, 48.6, 48.7, 48.8, 48.9, 49. , 49.1, 49.2, 49.3, 49.4, 49.5,
       49.6, 49.7, 49.8, 49.9])},
                   scoring='neg_mean_absolute_error')

In [108]:
rcv.best_params_

{'alpha': 48.1}

In [109]:
rcv.best_score_

-1.5146931290879562

In [110]:
best_ridge=rcv.best_estimator_

In [111]:
best_ridge.score(xtrain,ytrain)

0.5340483781074378

In [112]:
best_ridge.score(xtest,ytest)

0.517924077292121

### ESTIMATING HE VALUE

In [113]:
ypred_tr=best_ridge.predict(xtrain)
ypred_ts= best_ridge.predict(xtest)

In [119]:
x_new= pd.read_csv('test.csv')
x_new.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,74051,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,74052,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,74053,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,74054,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,74055,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


In [120]:
xnew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   num__Diameter        49368 non-null  float64
 1   num__Weight          49368 non-null  float64
 2   num__Shucked Weight  49368 non-null  float64
 3   num__Shell Weight    49368 non-null  float64
dtypes: float64(4)
memory usage: 1.5 MB


In [123]:
xnew=pre1.transform(x_new)
xnew

array([[-1.10363066, -1.16752768, -1.14759677, -1.11660983],
       [-0.57708136, -0.62286515, -0.54709872, -0.76860288],
       [-0.15584192, -0.69683166, -0.80950123, -0.53132541],
       ...,
       [ 0.5813271 ,  0.63232422,  0.2300164 ,  0.57596945],
       [-0.31380671, -0.61053739, -0.71362339, -0.49177916],
       [ 1.16053133,  1.04586429,  0.85574547,  1.05052439]])

In [124]:
xnew=pd.DataFrame(xnew,columns=cols2)
xnew

,num__Diameter,num__Weight,num__Shucked Weight,num__Shell Weight
0,-1.103631,-1.167528,-1.147597,-1.116610
1,-0.577081,-0.622865,-0.547099,-0.768603
2,-0.155842,-0.696832,-0.809501,-0.531325
3,-0.155842,0.394735,0.583251,0.085596
4,-0.735046,-0.918731,-0.814547,-0.946561
...,...,...,...,...
49363,0.054778,-0.558985,-0.607653,-0.827922
49364,-1.103631,-1.036405,-1.008826,-0.887242
49365,0.581327,0.632324,0.230016,0.575969
49366,-0.313807,-0.610537,-0.713623,-0.491779


In [126]:
preds= best_ridge.predict(xnew)

In [133]:
df_results= x_new[['id']]

In [134]:
df_results['Age']=preds

In [135]:
df_results

,id,Age
0,74051,7.793172
1,74052,8.215689
2,74053,10.042674
3,74054,8.702295
4,74055,7.972690
...,...,...
49363,123414,9.229327
49364,123415,8.072731
49365,123416,12.406708
49366,123417,9.762313


In [136]:
df_results.to_csv('Submission.csv',index=False)